## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\q8977\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#Conv+BN
classifier.add(Convolution2D(filters=32,kernel_size=(3,3),input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())
#Conv+BN
classifier.add(Convolution2D(filters=32,kernel_size=(3,3),activation='relu'))
classifier.add(BatchNormalization())
#Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100,activation='relu'))

#Output
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=20)

Train on 50000 samples
Epoch 1/20
50000/50000 [==============================] - 54s 1ms/sample - loss: 1.3879 - accuracy: 0.5170
Epoch 2/20
50000/50000 [==============================] - 56s 1ms/sample - loss: 0.9348 - accuracy: 0.6732
Epoch 3/20
50000/50000 [==============================] - 56s 1ms/sample - loss: 0.7577 - accuracy: 0.7355
Epoch 4/20
50000/50000 [==============================] - 56s 1ms/sample - loss: 0.6123 - accuracy: 0.7851
Epoch 5/20
50000/50000 [==============================] - 57s 1ms/sample - loss: 0.4733 - accuracy: 0.8367
Epoch 6/20
50000/50000 [==============================] - 53s 1ms/sample - loss: 0.3560 - accuracy: 0.8770
Epoch 7/20
50000/50000 [==============================] - 53s 1ms/sample - loss: 0.2533 - accuracy: 0.9133
Epoch 8/20
50000/50000 [==============================] - 54s 1ms/sample - loss: 0.1837 - accuracy: 0.9373
Epoch 9/20
50000/50000 [==============================] - 55s 1ms/sample - loss: 0.1498 - accuracy: 0.9496
Epoch 10/20
50

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.6269659e-16, 9.2814380e-29, 3.5505120e-11, 1.0000000e+00,
        4.1637041e-11, 4.5508360e-25, 1.6156577e-19, 6.7978425e-26,
        1.5653182e-16, 1.8262036e-27]], dtype=float32)